# Introduction to Generative AI

Generative AI is a type of artificial intelligence that can create new content and ideas, including conversations, stories, images, videos, and music. Like all artificial intelligence, generative AI is powered by machine learning models—very large models that are pre-trained on vast amounts of data and commonly referred to as Foundation Models (FMs). Apart from content creation, generative AI is also used to improve the quality of digital images, edit video, build prototypes quickly for manufacturing, augment data with synthetic datasets, and more.

## Using with Falcon model 

---
In this demo notebook, we demonstrate how to use the SageMaker Python SDK to deploy Falcon models for text generation. It is a permissively licensed ([Apache-2.0](https://jumpstart-cache-prod-us-east-2.s3.us-east-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt)) open source model trained on the [RefinedWeb dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb). We show several example use cases including code generation, question answering, translation etc.

---

### About the model

---
Falcon is a causal decoder-only model built by [Technology Innovation Institute](https://www.tii.ae/) (TII) and trained on more than 1 trillion tokens of RefinedWeb enhanced with curated corpora. It was built using custom-built tooling for data pre-processing and model training built on Amazon SageMaker. It features an architecture optimized for inference, with FlashAttention and multiquery. 


[Refined Web Dataset](https://huggingface.co/datasets/tiiuae/falcon-refinedweb): Falcon RefinedWeb is a massive English web dataset built by TII and released under an Apache 2.0 license. It is a highly filtered dataset with large scale de-duplication of CommonCrawl. It is observed that models trained on RefinedWeb achieve performance equal to or better than performance achieved by training model on curated datasets, while only relying on web data.

**Model Sizes:**
- **Falcon-7b**: It is a 7 billion parameter model trained on 1.5 trillion tokens.
- **Falcon-40B**: It is a 40 billion parameter model trained on 1 trillion tokens.  It has surpassed renowned models like LLaMA-65B, StableLM, RedPajama and MPT on the public leaderboard maintained by Hugging Face, demonstrating its exceptional performance without specialized fine-tuning. To see comparison, see [OpenLLM Leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard). 

**Instruct models (Falcon-7b-instruct/Falcon-40B-instruct):** Instruct models are base falcon models fine-tuned on a mixture of chat and instruction datasets. They are ready-to-use chat/instruct models.  To use these models, please select `model_id` in the cell above to be "huggingface-textgeneration-falcon-7b-instruct-bf16" or "huggingface-textgeneration-falcon-40b-instruct-bf16".

It is [recommended](https://huggingface.co/tiiuae/falcon-7b) that Instruct models should be used without fine-tuning and base models should be fine-tuned further on the specific task.

**Limitations:**

- Falcon models are mostly trained on English data and may not generalize to other languages. 
- Falcon carries the stereotypes and biases commonly encountered online and in the training data. Hence, it is recommended to develop guardrails and to take appropriate precautions for any production use. This is a raw, pretrained model, which should be further finetuned for most usecases.


## Step 1 : Upgrade SageMaker 

In [ ]:
# Skip this upgrade.
# !pip install sagemaker --quiet --upgrade --force-reinstall

## Step 2 : Define the Model
Define the **huggingface-llm-falcon-7b-instruct-bf16** model

In [14]:
model_id, model_version, = (
    "huggingface-llm-falcon-7b-bf16",
    "*",
)

## Step 3 : Deploy the model in Amazon SageMaker Inference Endpoint.


In [15]:
%%time
from sagemaker.jumpstart.model import JumpStartModel

my_model = JumpStartModel(model_id=model_id,instance_type="ml.g5.2xlarge" )
predictor = my_model.deploy()

Using model 'huggingface-llm-falcon-7b-bf16' with wildcard version identifier '*'. You can pin to version '4.1.11' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


-----------!CPU times: user 367 ms, sys: 8.75 ms, total: 375 ms
Wall time: 6min 3s


## Step 4 : Test the deployed model

The model supports a variety of actions like code generation, sentiment analysis, question answering, and summarization.

### Supported parameters

***
Some of the supported parameters while performing inference are the following:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **max_new_tokens:** Model generates text until the output length (excluding the input context length) reaches `max_new_tokens`. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **return_full_text:** If True, input text will be part of the output generated text. If specified, it must be boolean. The default value for it is False.
* **stop**: If specified, it must a list of strings. Text generation stops if any one of the specified strings is generated.

We may specify any subset of the parameters mentioned above while invoking an endpoint. 

For more parameters and information on HF LLM DLC, please see [this article](https://huggingface.co/blog/sagemaker-huggingface-llm#4-run-inference-and-chat-with-our-model).
***

### Step 4.1: Test endpoint

In [16]:
%%time


prompt = "Tell me about Amazon SageMaker."

payload = {
    "inputs": prompt,
    "parameters": {
        "do_sample": True,
        "top_p": 0.9,
        "temperature": 0.8,
        "max_new_tokens": 1024,
        "stop": ["<|endoftext|>", "</s>"]
    }
}

response = predictor.predict(payload)
print(response[0]["generated_text"])


Amazon SageMaker is a fully managed service for machine learning (ML) that enables data scientists and developers to quickly and easily build, train, and deploy ML models. Amazon SageMaker makes it easy to build and deploy ML models and take advantage of the latest ML techniques. Amazon SageMaker can be used to train and deploy machine learning models with code, libraries, and tools that you already know and use, such as Amazon SageMaker Studio, Amazon SageMaker Pipelines, and Amazon SageMaker Notebooks.
Amazon SageMaker automatically manages the infrastructure needed to build, train, and deploy ML models at any scale. You can focus on building, training, and deploying your models without worrying about managing infrastructure, capacity, and tuning algorithms for your specific needs. Amazon SageMaker is a fully managed service for ML. You can create a variety of models using a rich set of pre-trained machine learning models, or start from scratch and build models using your own code. 

### Step 4.2 : Test use cases

Let's define a function to query endpoint.

In [17]:
def query_endpoint(payload):
    """Query endpoint and print the response"""
    response = predictor.predict(payload)
    print(f"\033[1m Input:\033[0m {payload['inputs']}")
    print(f"\033[1m Output:\033[0m {response[0]['generated_text']}")

### Step 4.2.1 : Test code generation

In [18]:

payload = {"inputs": "Write a program to compute factorial in python:", "parameters":{"max_new_tokens": 200}}
query_endpoint(payload)

 Input: Write a program to compute factorial in python:
 Output: 
The factorial of a number is the product of all the numbers from 1 to the number.
For example, the factorial of 5 is 5 * 4 * 3 * 2 * 1 = 120.
The factorial of 10 is 10 * 9 * 8 * 7 * 6 * 5 * 4 * 3 * 2 * 1 = 3628800.
The factorial of 20 is 20 * 19 * 18 * 17 * 16 * 15 * 14 * 13 * 12 * 11 * 10 * 9 * 8 * 7 * 6 * 5 * 4 * 3 * 2 * 1 = 3628800


### Step 4.2.2 : Test sentence completion

In [19]:
payload = {
    "inputs": "Building a website can be done in 10 simple steps:",
    "parameters":{
        "max_new_tokens": 110,
        "no_repeat_ngram_size": 3
        }
}
query_endpoint(payload)

 Input: Building a website can be done in 10 simple steps:
 Output: 
1. Choose a domain name
2. Choose a web hosting provider
3. Choose a website builder
4. Choose a website template
5. Customize your website
6. Add content to your website
7. Add a contact form
8. Add a blog
9. Add a shopping cart
10. Promote your website
1. Choose a domain name
The first step to building a website is to choose a domain name. A domain name is the address of your website. It is the name that people will type into


### Step 4.2.3 : Test translation

In [20]:
payload = {
    "inputs": """Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>""",
    "parameters":{
        "max_new_tokens": 3
    }
}

query_endpoint(payload)

 Input: Translate English to French:

    sea otter => loutre de mer

    peppermint => menthe poivrée

    plush girafe => girafe peluche

    cheese =>
 Output:  fromage

   


### Step 4.2.4 : Test sentiment analysis

In [21]:
payload = {
    "inputs": """"I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:""",
    "parameters": {
        "max_new_tokens":2
    }
}
query_endpoint(payload)

 Input: "I hate it when my phone battery dies."
                Sentiment: Negative
                ###
                Tweet: "My day has been :+1:"
                Sentiment: Positive
                ###
                Tweet: "This is the link to the article"
                Sentiment: Neutral
                ###
                Tweet: "This new music video was incredibile"
                Sentiment:
 Output:  Positive
                


### Step 4.2.5 : Test question answering

In [22]:
payload = {
    "inputs": "Could you remind me when was the C programming language invented?",
    "parameters":{
        "max_new_tokens": 50
    }
}
query_endpoint(payload)

 Input: Could you remind me when was the C programming language invented?
 Output: 
The C programming language was invented in 1972 by Dennis Ritchie.
The C programming language was invented in 1972 by Dennis Ritchie.
The C programming language was invented in 1972 by Dennis Ritchie.
The


### Step 4.2.6 : Test recipe generation

In [23]:
payload = {"inputs": "What is the recipe for a delicious lemon cheesecake?", "parameters":{"max_new_tokens": 400}}
query_endpoint(payload)

 Input: What is the recipe for a delicious lemon cheesecake?
 Output: 
The recipe for a delicious lemon cheesecake is simple.
You will need:
- 1 cup of sugar
- 1 cup of butter
- 1 cup of flour
- 1 cup of lemon juice
- 1 cup of sour cream
- 1 cup of cream cheese
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon zest
- 1 cup of lemon juice
- 1 cup of lemon z

### Step 4.2.7 : Test summarization

In [ ]:
payload = {
    "inputs":"""Amazon SageMaker is a fully managed machine learning service. With SageMaker, 
    data scientists and developers can quickly and easily build and train machine learning models, 
    and then directly deploy them into a production-ready hosted environment. It provides an 
    integrated Jupyter authoring notebook instance for easy access to your data sources for 
    exploration and analysis, so you don't have to manage servers. It also provides common 
    machine learning algorithms that are optimized to run efficiently against extremely 
    large data in a distributed environment. With native support for bring-your-own-algorithms 
    and frameworks, SageMaker offers flexible distributed training options that adjust to your 
    specific workflows. Deploy a model into a secure and scalable environment by launching it 
    with a few clicks from SageMaker Studio or the SageMaker console. Summarize the article above:""",
    "parameters":{
        "max_new_tokens":200
        }
    }
query_endpoint(payload)

### 5. Clean up the endpoint - Do not run below codes.

In [ ]:
# Delete the SageMaker endpoint
# predictor.delete_model()
# predictor.delete_endpoint()